Установка библиотеки

In [ ]:
!pip install eyetrackfatigue


Работа с библиотекой

Сценарий первый: обработка входных данных, полученных от окулографа.


In [7]:
# подключение необходимых элементов библиотеки
from EyeTrackFatigue.Input import read_csv_file
from EyeTrackFatigue.Analise.ParsedData import ParsedData

section = read_csv_file('P5-23-01-25-crt1-morning.csv') # считывание файла с сырыми данными
# тестовые файлы расположенны в репозитории:
# выбранный файл необходимо разместить в среде выполнения
# https://github.com/AI-group-72/FAEyeTON/tree/main/TestData
metrics = ParsedData()
metrics.parse(section, 2, 1.0) # разметка данных, второй аргумент - 2 - метод по абсолютному расстоянию, размер области фиксации - 1.0
# 0 - по скорости, 1 - по радиусу области фиксации
metrics.calc_metrics() # рассчёт характеристик
metrics.to_csv('test1_output.csv', 'test') # вывод характеристик

Time column: 0 X column: 3 Y column: 4
parse...
marking fixations...
cleaning...
Pong = 1918 False Fix = 114
composing...
computing metrics...
fixations...
saccades...
calculating complete
csv file... test
forming a row... 
appending a row...
writen


Работа с библиотекой

Сценарий второй: загрузка ранее обученных моделей, оценка данных

In [12]:
import pandas as pd
# подключение необходимых элементов библиотеки
from EyeTrackFatigue.Evaluate.HybridEval import HybridEval
# загрузка данных - взят тестовый пример заранее посчитанных численных характеристик с репозитория
# файл необходимо разместить в среде выполнения
# https://github.com/AI-group-72/FAEyeTON/tree/main/TestData
data = pd.read_csv('dist_1.0.csv', delimiter=';')
data.drop(columns=['AU_cat'], inplace=True) # отделение выставленной оценки утомления
file_names = data.pop('File') # выделение наименований файлов
data.dropna()
data = (data - data.mean())/data.std() # нормализация данных

model = HybridEval()

# обучение моделей (с сохранением)
# model.edu(train_X, train_Y, test_X, test_Y, ['Basic', 'MLP', 'RFC', 'DCT']) # 'MLP', 'RFC', 'DCT'
# загрузка уже обученных моделей, по названию
# файлы с обученными моделями необходимо разместить в среде выполнения, в данном случае взяты из репозитория
model.load(['Basic', 'MLP' ]) # выбранны модели Basic -> алгоритмическая оценки и MLP - многослойный перцептрон

# оценка точности моделей
# print(model.eval(test_X, test_Y))
# оценка данных
_eval = model.evaluate(data)
data_eval = pd.DataFrame({'Eval' : _eval})
data_eval = pd.concat([file_names, data_eval], axis=1) # объединение оценённых данных с названиями файлов
data_eval.to_csv('eval_out.csv', sep=';', index=False)